In [1]:
import pandas as pd
import numpy as np
from astropy import units as u, constants as const
from tardis.io.decay import IsotopeAbundances
from pyne import nucname
from tardis.io.model_reader import read_csv_composition
import random

/home/jordi/conda/envs/tardis/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)


In [2]:
class TARDISModel():
    """The model in a format that works with TARDIS"""
    def __init__(self, time, velocity, density):
        self.time = time
        self.velocity = velocity
        self.density = density


In [3]:
class GenericModel():
    """A generic model that contains all properties"""
    def __init__(self, *properties, time = None):
        self.names = [tproperty.name for tproperty in properties]
        for tproperty in properties:
            setattr(self, tproperty.name, tproperty)
            max_property_time = u.Quantity(tproperty.time.max())
        
        if time is not None:
            try:
                time.to('s')
                
            except u.UnitsError:
                raise ValueError('"time" needs to be a quantity with units time (days, seconds, ...)')
        else:
            time = 0 * u.s
        
        self.time = max(time,max_property_time)
        
        for tproperty in properties:
            tproperty.time = self.time
        print(f"Model time set to {self.time}")
#different levels of verbosity would add warnings if the times passed to the model is lower than that of those in the properties
#Cannot assing time lower than x. Time set to x.
class BaseProperty():


    def __init__(self,time_0,name = None,quantity = None,units = None, **kwargs):
        if quantity is not None:
            try:
                quantity.to(units)
            except u.UnitConversionError:
                raise ValueError(f"'{name}' should have apropriate units ({units})")
        try:    
            time_0.to('s')
        except u.UnitConversionError:
            raise ValueError('"time" needs to be a quantity with units time (days, seconds, ...)')
        
        self.time_0 = time_0
        self.time = time_0
    
    def to_dataframe(self):
        """Parse BaseClass to a dataframe
         that includes properties in
         dataframe_dictionary and 
         user parameters added via `add_to_dictionary()`
         self.names_dictionary dictionary of base class
         self.user_dictionary dictionary of user added quntities
         self.dataframe : pd.DataFrame 
         """
        names = self.dataframe_dictionary
        dataframe = pd.DataFrame(dtype=np.float64)
        for prop in names:
            dataframe[names[prop]]=pd.Series(getattr(self,prop).value)
        for key,value in self.user_dictionary.items():
            dataframe[str(key)]=pd.Series(value)
        return dataframe


class Velocity(BaseProperty):
    """A class that holds the velocity arrays
        Parameters
        -------------
        velocity : astropy.units.Quantity
        can we change v_inner_boundry to velocity[0],
                      v_outer_boundry to velocity[-1]
        also why does radialmodel storabun = pd.read_csv('abun.csv' )

       dataframe_dictionary : dictionary of properties and its asociated column names. 
            eg. {'outer':'velocity_outer'}
         
    """
    name = 'velocity'
        
    def __init__(self, velocity_inner, velocity_outer, time_0 = 0 * u.d):
        try: 
            velocity_inner.to('cm/s')
            velocity_outer.to('cm/s')
        except:
            raise ValueError('"velocity" must be in units of velocity (km/s, cm/s,..), and i')
        self.inner = velocity_inner
        self._outer = velocity_outer
        BaseProperty.__init__(self, time_0)

        self.dataframe_dictionary = {'inner':'velocity_inner',
                                     'middle':'velocity_middle',
                                     'outer':'velocity_outer',
                                     'radius':'radius',
                                     'volume':'volume'}

    @property
    def no_of_cells(self):
        return len(self.inner)

    @property
    def outer(self):
        return self._outer

    @property
    def middle(self):
        return (0.5 * (self.inner + self.outer)).cgs
    
    @property 
    def radius(self):
        return (self.time * self.outer).cgs

    @property
    def volume(self):
        volume = (4./3 * np.pi * (self.outer**3 - self.inner**3) * self.time**3).cgs
        return volume
   


            
class Density(BaseProperty):
    """A class that holds an initial density and time
    Parameters
    ----------
    density_0 : astropy.units.Quantity Density at time_0
    time_0 : astropy.units.Quantity Time for initial density. Defaults to 1s to avoid `inf`
    current : astropy.units.Quantity Density at a given time. 
    It only makes sense if self.time is updated 
    electron? : astropy.units.Quantity electron densities
    """
    name = 'density'
    
    def __init__(self, density, time_0 = 1 * u.s):
        BaseProperty.__init__(self, time_0, self.name, density, 'g/cm**3')
        self.density_0 = density.cgs
        dataframe_dictionary = {'current' : 'density'}
        
    @property
    def current(self):
        return (self.density_0 * (self.time/self.time_0)**-3).cgs

    @property
    def initial(self):
        return self.density_0

    @classmethod
    def from_exponential(cls, rho_0, velocity_object, exponent, time_0):
      density = rho_0 * np.exp(-(velocity_object.middle / velocity_object.inner[0]))
      return cls(density, time_0)

class Abundances(BaseProperty):
    """A class for abundances and isotope abundances. It takes dataframes for elemental abundance and isotopic. See tardis.io.decay.IsotopicAbundances for more info

    """
    name = 'abundance'          
    
    def __init__(self, elemental, isotope, time_0):
        BaseProperty.__init__(self, time_0)
        self._elemental_0 = elemental.sort_values(by='atomic_number',axis=0)
        self._isotope_0 = IsotopeAbundances(isotope)


    @property
    def _isotope(self):
        return self._isotope_0.decay(self.time)

    @property
    def _elemental(self):
        return self._isotope_0.decay(self.time).merge(self._elemental_0)
    
    @property
    def isotope(self):
        isotope_buffer = self._isotope
        atomic_index = isotope_buffer.index.get_level_values('atomic_number')
        mass_index = isotope_buffer.index.get_level_values('mass_number')
        isotope_buffer.index = self.get_isotope_index(atomic_index,mass_index)
        isotope_buffer.name = 'isotope'
        isotope_buffer.columns.name = 'cell'
        return isotope_buffer
        
    @property
    def elemental(self):
        elemental_buffer = self._elemental
        elemental_buffer.index = self.get_elemental_index(elemental_buffer.index)
        elemental_buffer.index.name = 'element'
        elemental_buffer.columns.name = 'cell'
        return elemental_buffer
    
    def get_elemental_index(self,number_index):
        names_index=[]
        for number in number_index:
            names_index.append(nucname.name(number))
        return names_index
    
    def get_isotope_index(self,atomic_number_index, mass_number_index):
        names_index=[]
        for atomic_number,mass_number in zip(atomic_number_index, mass_number_index):
            names_index.append('{0:s}{1:d}'.format(nucname.name(atomic_number), mass_number))
        return names_index
    
class RadiationField(BaseProperty):
    """ A class for radiative temperature and dilution factor
        
    Attributes
    ----------
        
    
    """
    name = 'radiation_field'
    def __init__(self,radiative_temperature = None,dilution_factor = None, time_0 = 0 * u.d):
        BaseProperty.__init__(self, time_0, self.name, radiative_temperature, 'K')
    
        self.radiative_temperature = radiative_temperature
        self.dilution_factor = dilution_factor
        
    
    


### We can use SN toy model 6 to check functionality

In [4]:
toy_model = pd.read_csv('snia_toy06.dat', index_col='idx', comment = '#', delimiter='\s+')
index,abundances,isotope_abundances = read_csv_composition('toy06_ab.csv')
temp = toy_model['temp[K]'].to_numpy() * u.K
vel = toy_model['vel[km/s]'].to_numpy()
vel_in=np.append(0,vel[:-1]) * u.km/u.s
vel_ou=vel * u.km/u.s
den =toy_model['dens[gcc]'].to_numpy() * u.g/u.cm**3
abund = Abundances(abundances, isotope_abundances, 2 * u.d)
model = GenericModel(Velocity(vel_in,vel_ou),abund,Density(den),RadiationField(temp),time = 2 * u.d)


Model time set to 2.0 d


## We can start the model with any amount of Properties. Every class checks if units are correct

In [7]:
model.density.initial

<Quantity [6.2419e-10, 6.1502e-10, 6.0492e-10, 5.9475e-10, 5.8468e-10,
           5.7473e-10, 5.6493e-10, 5.5529e-10, 5.4581e-10, 5.3648e-10,
           5.2731e-10, 5.1829e-10, 5.0942e-10, 5.0071e-10, 4.9214e-10,
           4.8372e-10, 4.7544e-10, 4.6731e-10, 4.5931e-10, 4.5145e-10,
           4.4372e-10, 4.3612e-10, 4.2866e-10, 4.2132e-10, 4.1411e-10,
           4.0702e-10, 4.0005e-10, 3.9320e-10, 3.8647e-10, 3.7985e-10,
           3.7335e-10, 3.6695e-10, 3.6067e-10, 3.5450e-10, 3.4843e-10,
           3.4246e-10, 3.3660e-10, 3.3083e-10, 3.2517e-10, 3.1960e-10,
           3.1413e-10, 3.0875e-10, 3.0346e-10, 2.9827e-10, 2.9316e-10,
           2.8814e-10, 2.8321e-10, 2.7836e-10, 2.7359e-10, 2.6891e-10,
           2.6430e-10, 2.5977e-10, 2.5533e-10, 2.5095e-10, 2.4666e-10,
           2.4243e-10, 2.3828e-10, 2.3420e-10, 2.3019e-10, 2.2625e-10,
           2.2238e-10, 2.1857e-10, 2.1483e-10, 2.1115e-10, 2.0753e-10,
           2.0398e-10, 2.0048e-10, 1.9705e-10, 1.9368e-10, 1.9036e-10,
      

### Velocity

#### Inner velocity and outer velocity arrays are now needed to start the Velocity Object

In [39]:
model.velocity.inner

<Quantity [0.0000e+00, 2.5000e+01, 7.5000e+01, 1.2500e+02, 1.7500e+02,
           2.2500e+02, 2.7500e+02, 3.2500e+02, 3.7500e+02, 4.2500e+02,
           4.7500e+02, 5.2500e+02, 5.7500e+02, 6.2500e+02, 6.7500e+02,
           7.2500e+02, 7.7500e+02, 8.2500e+02, 8.7500e+02, 9.2500e+02,
           9.7500e+02, 1.0250e+03, 1.0750e+03, 1.1250e+03, 1.1750e+03,
           1.2250e+03, 1.2750e+03, 1.3250e+03, 1.3750e+03, 1.4250e+03,
           1.4750e+03, 1.5250e+03, 1.5750e+03, 1.6250e+03, 1.6750e+03,
           1.7250e+03, 1.7750e+03, 1.8250e+03, 1.8750e+03, 1.9250e+03,
           1.9750e+03, 2.0250e+03, 2.0750e+03, 2.1250e+03, 2.1750e+03,
           2.2250e+03, 2.2750e+03, 2.3250e+03, 2.3750e+03, 2.4250e+03,
           2.4750e+03, 2.5250e+03, 2.5750e+03, 2.6250e+03, 2.6750e+03,
           2.7250e+03, 2.7750e+03, 2.8250e+03, 2.8750e+03, 2.9250e+03,
           2.9750e+03, 3.0250e+03, 3.0750e+03, 3.1250e+03, 3.1750e+03,
           3.2250e+03, 3.2750e+03, 3.3250e+03, 3.3750e+03, 3.4250e+03,
      

#### As a small verification radius from the csv model coincides with the one calculated by the model. Not all of them match, I should check why, there seems to be a pattern there.

In [ ]:
model.velocity.radius.value - toy_model['rad[cm]'].to_numpy()

### Density

#### `density.current` drops with time

#### Default value for `density.time_0` is 1s to prevent infinte values

In [ ]:
model.density.current

#### initial density is stored in `density.initial` 

In [ ]:
model.density.initial

###  Radiative temperature is stored inside the Radiation Field object

In [ ]:
model.radiation_field.radiative_temperature

###  Abundances are evolved on demand so models can be tried at different times

##### `abundance.elemental` shows abundance already decayed

In [ ]:
model.abundance.elemental

#### `_elemental` stores initial abundance

In [ ]:
model.abundance._elemental_0

#### The dataframes with number indeces are not discarded, but stored as `_elemental` and `_isotopic`

In [14]:
model.abundance._elemental

,0,1,2,3,4,5,6,7,8,9,...,797,798,799,800,801,802,803,804,805,806
atomic_number,,,,,,,,,,,,,,,,,,,,,
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
14,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.55,0.55,0.55,0.55,0.55,0.55,0.55,0.55,0.55,0.55
16,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.35,0.35,0.35,0.35,0.35,0.35,0.35,0.35,0.35,0.35
20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10
26,0.001889,0.001889,0.001889,0.001889,0.001889,0.001889,0.001889,0.001889,0.001889,0.001889,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
27,0.202139,0.202139,0.202139,0.202139,0.202139,0.202139,0.202139,0.202139,0.202139,0.202139,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
28,0.795972,0.795972,0.795972,0.795972,0.795972,0.795972,0.795972,0.795972,0.795972,0.795972,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


### `isotope` shows decayed isotope abundances

In [12]:
model.abundance.isotope

cell,0,1,2,3,4,5,6,7,8,9,...,797,798,799,800,801,802,803,804,805,806
Fe56,0.001503,0.001503,0.001503,0.001503,0.001503,0.001503,0.001503,0.001503,0.001503,0.001503,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Co56,0.160893,0.160893,0.160893,0.160893,0.160893,0.160893,0.160893,0.160893,0.160893,0.160893,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ni56,0.633566,0.633566,0.633566,0.633566,0.633566,0.633566,0.633566,0.633566,0.633566,0.633566,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### `_isotope_0` stores initial isotope abundance

In [11]:
model.abundance._isotope_0

,,0,1,2,3,4,5,6,7,8,9,...,797,798,799,800,801,802,803,804,805,806
atomic_number,mass_number,,,,,,,,,,,,,,,,,,,,,
28,56,0.79597,0.79597,0.79597,0.79597,0.79597,0.79597,0.79597,0.79597,0.79597,0.79597,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
v = Velocity(vel_in,vel_ou)

In [13]:
a = 1 * u.d
b='s'

In [14]:
a.to(b)

<Quantity 86400. s>

In [25]:
a = 10 * u.K

In [26]:
a.cgs

<Quantity 10. K>